# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Откройте файл с данными и изучите общую информацию

Приступим к работе с файлом. Прочитаем его, проверим данные о столбцах и получим первые десять строк таблицы.

In [1]:
import pandas as pd
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**
Все значения в столбцах готовы к обработке, столбцы с числами имеют формат int или float.

В столбце с днями работы есть отрицательные дробные значения - подозрительно и непонятно. Требуется понять, как их обработать, и почему это появилось. 

Есть пропуски в значениях отработанных дней и зарплаты - необходимо понять, откуда они.

В столбце "образование" использовался разный регистр при вводе данных - обработать.

## Предобработка данных

### Обработка пропусков

Во-первых, надо разобраться с отрицательными значениями в столбце "отработанные дни" и "количество детей". Заменим отрицательные значения положительными с помощью метода apply(abs). И предположим, что произошла ошибка, из-за которой в некоторых случаях в столбце указаны не отработанные дни, а отработанные часы. Переведём эти значения в часы.

In [3]:
data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int64)

0 в столбце с возрастом похож на пропуск.

In [4]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В таблице возраста у 101 человека возраст составляет 0 лет. Исправим это.

In [5]:
years_median = data.groupby('income_type')['dob_years'].median()

for income_type in years_median.index: 
    data.loc[data['income_type'] == income_type,'dob_years'] =  data.loc[data['income_type'] == income_type,'dob_years'].replace(0, years_median.loc[income_type])  

In [6]:
data['dob_years'].sort_values().unique()

array([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75], dtype=int64)

С возрастом разобрались. Посмотрим, что там по детям

In [7]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Теперь с количеством детей всё в порядке. Хотя, конечно, согласна, что 20 детей - это перебор. Их немного, поэтому допущу, что тут имеет место человеческий фактор: опечатка. Было 2, стало 20

In [8]:
data.loc[data['children'] == 20, 'children'] = 2

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4,  5], dtype=int64)

In [10]:
#заменим отрицательное значение на значение по модулю
data['children'] = data['children'].apply(abs)

In [11]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Отрицательных дней работы быть не должно, у нас всё-таки не рабство. Поэтому предположим, что имеет место ошибка в данных.

In [12]:
#заменим отрицательные значения на значения по модулю
data['days_employed'] = data['days_employed'].apply(abs)

In [13]:
#если отработанных значений больше 30000 примем их за часы и переведём в дни
data.loc[data['days_employed'] >30000, 'days_employed'] = data.loc[data['days_employed'] >30000, 'days_employed'] / 24

In [14]:
data.sort_values(by='days_employed', ascending=False).head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
6954,0,16739.808353,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,16738.158823,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,16736.462226,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,16736.436110,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,16735.993752,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
4697,0,16734.793029,56,среднее,1,женат / замужем,0,F,пенсионер,0,48242.322502,покупка недвижимости
13420,0,16734.151387,63,Среднее,1,гражданский брак,1,F,пенсионер,0,51449.788325,сыграть свадьбу
17823,0,16733.936484,59,среднее,1,женат / замужем,0,F,пенсионер,0,152769.694536,покупка жилья для сдачи


Получилось, что к 61 году человек отработал 50 лет. Многовато, конечно. Но если предположить, что его 18000 - это часы, тогда его трудовой стаж составит 2 года, а это ещё более странно. Оставим эти значения в днях.

Теперь разберёмся с пропущенными значениями в столбцах "отработанные дни" и "з/п". Посмотрим, сколько у на всего пропусков и где они расположены.

In [15]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [16]:
data[data['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Значения пропущены не только у предположительно неработающих пенсионеров. Значит, заменять их нулями будет неверно. 

Вероятно, ошибка техническая. Человек ошибался бы в разных столбцах при заполнении данных. И вряд ли человек делал бы абсолютно одинаковую ошибку: пропускал значения сразу в двух столбцах. Возможно, значения не соответствовали нужному формату, поэтому были удалены.

Найдём для каждого столбца медианное значение и заменим им значение отсутствующее.

In [17]:
income_median = data.groupby('income_type')['total_income'].median()

for income_type in income_median.index: 
    data.loc[data['income_type'] == income_type,'total_income'] = data.loc[data['income_type'] == income_type,'total_income'].fillna(income_median.loc[income_type]) 

In [18]:
days_median = data.groupby('income_type')['days_employed'].median()

for income_type in days_median.index: 
    data.loc[data['income_type'] == income_type,'days_employed'] = data.loc[data['income_type'] == income_type,'days_employed'].fillna(income_median.loc[income_type]) 

In [19]:
days_median

income_type
безработный        15267.235531
в декрете           3296.759962
госслужащий         2689.368353
компаньон           1547.382223
пенсионер          15217.221094
предприниматель      520.848083
сотрудник           1574.202821
студент              578.751554
Name: days_employed, dtype: float64

In [20]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [21]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**
Все пропущенные значения в таблице я успешно заменила на медианные значения по столбцу. Пропусков в таблице больше нет.


Отрицательные значения отработанных дней поменяла на положительные. Предположив, что значения больше 30 000 - это часы, заменила их на дни.

Отрицательных детей и нулевой возраст тоже исправила. Всё-таки отрицательных детей быть не должно.

### Замена типа данных

Если запись зарплаты в виде десятичного числа ещё возможна, то дробное количество отработанных дней явно вызывает сомнения. Переведём значения столбца days_employed в формат int методов astype(). Метод to_numeric() переведёт данные в формат float. Нам это не нужно.

In [22]:
data['days_employed'] = data['days_employed'].astype('int')

In [23]:
data['total_income'] = data['total_income'].astype('int')

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int32
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int32
purpose             21525 non-null object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


In [25]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**
Я заменила дробные значения на целочисленные. Получилось, что некоторые отработали очень много, другие - очень мало. Возможно, у них не было официального трудоустройства.

### Обработка дубликатов

Во-первых, сразу разберёмся с регистром. Приведём все строки в столбце "Образование" к нижнему регистру.

In [26]:
data['education'] = data['education'].str.lower()

Найдём и удалим грубые дубликаты методами duplicated() и drop_duplicates() в комбинации с reset_index().

In [27]:
data.duplicated().sum()

71

In [28]:
data = data.drop_duplicates().reset_index(drop=True)

In [29]:
data.duplicated().sum()

0

In [30]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [31]:
data['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

In [32]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [33]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**
Предполагаю, что дубликаты в самих данных могли появиться потому, что люди заполняли заявление несколько раз. На всю таблицу выявился всего 71 дубликат. Из 20000 значений количество дубликатов - незначительное.

Остался разброд в целях, например, "образование" и "получение образования", "автомобили" и "автомобиль". Они явно появились, потому что люди заполняли эту графу самостоятельно. У них не было унифицированных вариантов ответа, например, выпадающего списка. Люди писали, что думали. С этими дубликатами будем разбираться дальше.

### Лемматизация

Нам необходимо лемматизировать цели кредита. Так мы сможем увидеть, на какие цели кредит берут чаще всего и соотнести цели кредита с категориями граждан. Воспользуемся библиотекой pymystem3 и методом lemmatize(), а так же контейнером Counter из модуля collections.

In [34]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
text = data['purpose']
lemmas = ' '.join(text)
lemmas = m.lemmatize(lemmas)
Counter(lemmas)

Counter({'покупка': 5897,
         ' ': 55023,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

Из полученного списка выделим основные категории: покупка автомобиля -> автомобиль, свадьба, образование, ремонт и приобретение недвижимости -> недвижимость. Пора переходить к категоризации.

**Вывод** Самые частые цели кредита - покупка автомобиля, свадьба, образование и операции с недвижимостью.

<div class="alert alert-success">
<b>Комментарий ревьюера✅ v2: </b> Сейчас все отлично! Кстати, вот как это можно было сделать покороче:

```python
Counter(data['purpose'].apply(m.lemmatize).sum())
```
</div>

### Категоризация данных

Категоризируем данные по целям кредита. Для этого напишем функцию purpose_categorize(purpose), которая соотнесет все цели кредита, указанные в таблице по выделенным мной четырём категориям. В четырёх категориях кредитов разбираться проще, чем в 20.

"Цели кредита" - это единственный столбец, который люди заполняли вручную. Там много одинаковых целей, написанных разными фразами, их необходимо категоризировать. Все остальные данные в столбцах унифицированы.

In [35]:
def purpose_categorize(purpose):
    purpose_lemmas = m.lemmatize(purpose)
    if 'недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas:
        return 'недвижимость'
    if 'свадьба' in purpose_lemmas:
        return 'свадьба'
    if 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    if 'образование' in purpose_lemmas:
        return 'образование'
    else:
        'Цель не определена'

Применим нашу функцию к столбцу "цели кредита". Заменим значения в столбце purpose на значения, полученные в результате работы функции. Не вижу смысла создавать новый столбец, который будет полностью повторять уже существующий.

In [ ]:
data['purpose'] = data['purpose'].apply(purpose_categorize)

In [ ]:
data.head()

In [ ]:
data['purpose'].unique()

**Вывод на данном этапе**

Я убила двух зайцев одним ударом: избавилась от дубликатов целей для кредита и категоризировала заявки.

Разделим категории граждан по уровню дохода. Возьмём доход менее 50000 малым, от 50000 до 150000 - средним, от 150000 до 400000 - высоким, всё остальное - отличным. Напишем функцию income_categorize(total_income), которая разделит наших граждан на категории.

In [ ]:
def income_categorize(total_income):
    if total_income <= 50000:
        return 'Малый доход'
    if (total_income > 50000) & (total_income <= 150000):
        return 'Средний доход'
    if total_income > 150000:
        return 'Высокий доход'

In [ ]:
#создаём новый столбец income_category, применив функцию к столбцу total_income
data['income_category'] = data['total_income'].apply(income_categorize)

In [ ]:
data.head()

3. Разделим наших граждан на категории по возрасту. Самое маленькое значение возраста в таблице - 19. Значит, детей брать не будем. Создадим функцию years_categorize(dob_years) и разделим всех наших граждан на три условные категории: взрослые, пенсионеры и молодёжь.

In [ ]:
def years_categorize(dob_years):
    if dob_years <= 33:
        return 'Молодёжь'
    if (dob_years > 33) & (dob_years <= 65):
        return 'Взрослый'
    if dob_years > 65:
        return 'Пенсионер'

In [ ]:
#создадим новый столбец years_category, применив функцию к столбцу dob_years
data['years_category'] = data['dob_years'].apply(years_categorize)

In [ ]:
data.tail(10)

**Вывод**

Неопознанных целей для получения кредита нет, всё разлетелось по категориям. Доход упорядочен, как и возраст.

Теперь, когда все данные приняли единообразный вид, можно отвечтать на основные вопросы работы.

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

1. Узнаем, как связаны количество детей и возврат кредита в срок. Воспользуемся функцией pivot_table() и создадим сводную таблицу, в которой сгруппируем данные по детям, и узнаем, каков процент должников.

In [ ]:
child_debt = data.pivot_table(index=['children'], values='debt', aggfunc=['sum', 'count', 'mean'])
child_debt.columns = ['sum', 'total', '%']
child_debt['%'] = child_debt['%'] * 100
child_debt = child_debt.sort_values(by=['%'], ascending=False)
child_debt

**Вывод**

Глядя на таблицу, напрашивается один вывод: чем меньше детей, тем меньше задолженностей по кредиту. Картину портят только люди, у которых 5 детей. Но их всего 9 человек, можно не учитывать.

Две самые многочисленные группы: семьи без детей и семьи с одним ребенком. Они отличаются всего на 1,5%. С другой стороны, если бы количество семей без детей с задолженностями по кредиту увеличилось на 1,5%, их абсолютно значение составило бы 1260 человек, вместо 1060. Прибавка в 200 человек. 

Получается, почти каждый десятый человек с 1-2 детьми имеет задолженность по кредиту. По-моему, это достаточно много.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

2. Узнаем, как связано семейное положение граждан с возвратом кредита в срок. Создадим сводную таблицу:

In [ ]:
family_status_debt = data.pivot_table(index=['family_status'], values='debt', aggfunc=['sum', 'count', 'mean'])
family_status_debt.columns = ['sum', 'total', '%']
family_status_debt['%'] = family_status_debt['%'] * 100
family_status_debt = family_status_debt.sort_values(by=['%'], ascending=False)
family_status_debt

Меньше всего задолженностей у вдовцов. И теперь непонятно: радоваться за них или грустить. 

Почти 10% людей, не живущих в официальном браке имеют задолженность по кредиту. Создание семьи явно прибавляет людям ответственности.

С другой стороны, интересно посмотреть на ответственность отдельно одиноких людей и людей с супругом.

In [ ]:
def family_status_category(family_status_id):
    if family_status_id == 0 or family_status_id == 1:
        return 'Семьянин'
    if family_status_id == 2 or family_status_id == 3 or family_status_id == 4:
        return 'Одиночка'
    

In [ ]:
data['family'] = data['family_status_id'].apply(family_status_category)
data.tail(10)

In [ ]:
family_debt = data.pivot_table(index=['family'], values='debt', aggfunc=['sum', 'count', 'mean'])
family_debt.columns = ['sum', 'total', '%']
family_debt['%'] = family_debt['%'] * 100
family_debt = family_debt.sort_values(by=['%'], ascending=False)
family_debt

**Вывод**

Могу утверждать, что семейное положение влияет на количество взятых кредитов и не влияет на количество задолженностей.


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

3. Узнаем, как связан уровень дохода граждан с возвратом кредита в срок. Создадим сводную таблицу:

In [ ]:
income_debt = data.pivot_table(index=['income_category'], values='debt', aggfunc=['sum', 'count', 'mean'])
income_debt.columns = ['sum', 'total', '%']
income_debt['%'] = income_debt['%'] * 100
income_debt = income_debt.sort_values(by=['%'], ascending=False)
income_debt

**Вывод** Люди с малым доходом явно внимательнее к деньгам и погашениям задолженностей, чем люди с нормальным доходом.

- Как разные цели кредита влияют на его возврат в срок?

4. Узнаем, как связаны цели кредита с возвратом его в срок. Создадим сводную таблицу:

In [ ]:
purpose_debt = data.pivot_table(index=['purpose'], values='debt', aggfunc=['sum', 'count', 'mean'])
purpose_debt.columns = ['sum', 'total', '%']
purpose_debt['%'] = purpose_debt['%'] * 100
purpose_debt = purpose_debt.sort_values(by=['%'], ascending=False)
purpose_debt

**Вывод** Между самой большой и самой маленькой категорией должников разница 2%.

5. Хочу добавить к сравнению ещё и возраст заёмщиков.

In [ ]:
years_debt = data.pivot_table(index=['years_category'], values='debt', aggfunc=['sum', 'count', 'mean'])
years_debt.columns = ['sum', 'total', '%']
years_debt['%'] = years_debt['%'] * 100
years_debt = years_debt.sort_values(by=['%'], ascending=False)
years_debt

Вот оно как! Пенсионеры - самые молодцы. Взрослое население укладывается в те самые средние 7%, а больше всего должников среди молодёжи! Так я и думала.

## Общий вывод

Судя по нашим данным, больше всего кредитов берут молодые семьи без детей на покупку недвижимости. И больше всего должников среди этих же самых людей. 

Первая гипотеза о связи количества детей с количеством задолженностей по кредиту подтвердилась: таблица показывает, что чем больше детей, тем чаще люди имеют задолженности. Связь есть

Вторая гипотеза о связи семейного положения с количеством задолженностей не подтвердилась: люди, живущие в паре, чаще берут кредиты, но проблем с задолженностями у одиноких людей и людей в отношениях примерно одинаковое количество. Связи нет.

Третья гипотеза о связи уровня доходов и количества задолженностей подтвердилась: люди с маленьким доходом задолженностей стараются не допускать. Связь есть.

Четвёртая гипотеза о связи целей кредита с задолженностями подтвердилась: чем более устойчивые цели, тем более устойчивые платежи. Люди, взявшие кредит на недвижимость допускают меньше задолженностей, чем люди, взявшие кредит на автомобиль.

Пятая, моя гипотеза, тоже подтвердилась. Я предположила, что люди разных возрастных категорий тоже возвращают кредит по-разному, и это оказалось правдой: у молодёжи существуют задолженности по кредитам в 11% случаев.
